## Calculating time domain features

In this notebook, we will calculate time domain features from raw data and form a feature matrix. We will then compare the computed feature matrix with the one that is available online. To compute feature matrix, we will first segment our data into segments of length 2048. There is no overlap between consecutive segments. We will then compute time domain features for each segment. We will use the already segmented raw time domain data available [here](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/data/CWRU_48k_load_1_CNN_data.npz). For details about data description and segmentation procedure, see [this notebook](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/CWRU_time_domain_data_preprocessing.ipynb).

We have taken segment length of 2048 because the feature matrix that is available online has been computed using 2048 as segment length. As we are going to compare our computed result with the feature matrix that is available online, we will also take the same segment length. But there is nothing special with the number 2048. We could have taken 1024 as our segment length. The only criteria segment length has to satisfy is that within the segment our event of interest (i.e., fault signature) should occur. 

In [1]:
import numpy as np
import pandas as pd

Download the raw time domain data from [here](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/data/CWRU_48k_load_1_CNN_data.npz) and read it.

In [2]:
file = np.load("./data/CWRU_48k_load_1_CNN_data.npz")
file.files

['data', 'labels']

In [3]:
data = file["data"]
labels = file["labels"]

In [4]:
data.shape

(4600, 32, 32)

We will resize the data so that our segment length is 2048.

In [5]:
resized_data = np.reshape(data, (2300,2048))
resized_data.shape

(2300, 2048)

## Calculate features

We will calculate following time domain features.

* Maximum value
* Minimum value
* Mean value 
* Standard deviation (Unbiased standard deviation)
* Root mean square value (RMS)
* Skewness
* Kurtosis
* Crest factor = $\frac{\text{Maximum value}}{\text{RMS}}$
* Form factor = $\frac{\text{RMS}}{\text{Mean value}}$

The only two statistical features that require some comment are `skewness` and `kurtosis`. In literature, there are different definitions for both. We will use the following two definitions.

$$\text{skewness} = \frac{\text{Third moment}}{{\text{(Std. deviation)}}^3} = \frac{ \frac{\Sigma_{i = 1}^{N}{(x_i - \mu)^3}}{N}} {\text{(Std. deviation)}^3}$$

where, $\mu$= mean and **standard deviation is the unbiased standard deviation**. Similarly `kurtosis` is defined as:

$$\text{kurtosis} = \frac{\text{Fourth moment}}{{\text{(Std. deviation)}}^4}-3 = \frac{ \frac{\Sigma_{i = 1}^{N}{(x_i - \mu)^4}}{N}} {\text{(Std. deviation)}^4} - 3$$

Using `scipy.stats` we can also compute `skewness` and `kurtosis`. But their default definitions are slightly different in `scipy`. Therefore, we will not use `scipy`. Rather, we will manually compute those 2 features. 

### Why have I not used the default `scipy` definition?

When I started this project, I used to use `R` for data analysis (I still use `R`). In `R`, there is a package `e1071` that has functions to compute `skewness` and `kurtosis`. Default versions of these functions use the above formulae to calculate `skewness` and `kurtosis`. The feature matrix that is available online is created using these functions. Now, as we intend to reproduce our feature matrix in `Python`, we have to use the same formulae that was used to compute it at the first place. This is the reason behind our departure from `scipy` definition.

In [6]:
def compute_skewness(x):
    """x should be 1D np array."""
    N = len(x)
    third_moment = np.sum((x - np.mean(x))**3) / N
    s_3 = np.std(x, ddof = 1) ** 3
    return third_moment/s_3

In [7]:
def compute_kurtosis(x):
    """x should be 1D np array."""
    N = len(x)
    fourth_moment = np.sum((x - np.mean(x))**4) / N
    s_4 = np.std(x, ddof = 1) ** 4
    return fourth_moment / s_4 - 3

In [8]:
feature_matrix_time = np.repeat(np.nan, 2300*9).reshape((2300,9))
N = resized_data.shape[1]
for i in np.arange(resized_data.shape[0]):
    temp = resized_data[i,:]
    feature_matrix_time[i,0] = np.max(temp)
    feature_matrix_time[i,1] = np.min(temp)
    feature_matrix_time[i,2] = np.mean(temp)
    feature_matrix_time[i,3] = np.std(temp, ddof = 1)
    feature_matrix_time[i,4] = np.sqrt(np.mean(temp ** 2))
    feature_matrix_time[i,5] = compute_skewness(temp)
    feature_matrix_time[i,6] = compute_kurtosis(temp)
    feature_matrix_time[i,7] = feature_matrix_time[i,0]/feature_matrix_time[i,4]
    feature_matrix_time[i,8] = feature_matrix_time[i,4]/feature_matrix_time[i,2]

Now read the online feature_matrix using the online link.

In [9]:
link = "https://raw.githubusercontent.com/biswajitsahoo1111/cbm_codes_open/master/notebooks/data/feature_time_48k_2048_load_1.csv"
online_feature_matrix = pd.read_csv(link)
online_feature_matrix.head()

,max,min,mean,sd,rms,skewness,kurtosis,crest,form,fault
0,0.35986,-0.41890,0.017840,0.122746,0.124006,-0.118571,-0.042219,2.901946,6.950855,Ball_007_1
1,0.46772,-0.36111,0.022255,0.132488,0.134312,0.174699,-0.081548,3.482334,6.035202,Ball_007_1
2,0.46855,-0.43809,0.020470,0.149651,0.151008,0.040339,-0.274069,3.102819,7.376926,Ball_007_1
3,0.58475,-0.54303,0.020960,0.157067,0.158422,-0.023266,0.134692,3.691097,7.558387,Ball_007_1
4,0.44685,-0.57891,0.022167,0.138189,0.139922,-0.081534,0.402783,3.193561,6.312085,Ball_007_1


In [10]:
online_feature_matrix.shape

(2300, 10)

In [11]:
2300*9

20700

Compare the values of computed feature matrix with the one available online.

In [12]:
np.sum(np.isclose(feature_matrix_time, online_feature_matrix.iloc[:,:9]))

20700

Now we can use our feature matrix for further analysis. For an example, see [this notebook](https://github.com/biswajitsahoo1111/cbm_codes_open/blob/master/notebooks/SVM_multiclass_time_cwru_python.ipynb) where we have used Support Vector Machine (SVM) on time domain feature matrix for multiclass fault classification. We achieved an overall classification accuracy of 96.5%.